In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_data = 5000   # number of data
num_vars = 5      # number of decision variables
num_ints = 5      # number of integer decision variables
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# parameters as input data
p_train = np.random.uniform(2.0, 6.0, (train_size, 1)).astype(np.float32)
a_train = np.random.uniform(6.0, 15.0, (train_size, num_vars)).astype(np.float32)
p_test = np.random.uniform(6.0, 15.0, (test_size, 1)).astype(np.float32)
a_test = np.random.uniform(0.2, 1.2, (test_size, num_vars)).astype(np.float32)
p_dev = np.random.uniform(6.0, 15.0, (val_size, 1)).astype(np.float32)
a_dev = np.random.uniform(0.2, 1.2, (val_size, num_vars)).astype(np.float32)

In [5]:
# nm datasets
from neuromancer.dataset import DictDataset
data_train = DictDataset({"p":p_train, "a":a_train}, name="train")
data_test = DictDataset({"p":p_test, "a":a_test}, name="test")
data_dev = DictDataset({"p":p_dev, "a":a_dev}, name="dev")

In [6]:
# torch dataloaders
from torch.utils.data import DataLoader
loader_train = DataLoader(data_train, batch_size=32, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size=32, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size=32, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=True)

## NM Problem

In [7]:
import neuromancer as nm
from problem.neural import probRastrigin

def getNMProb(round_module):
    # parameters
    p = nm.constraint.variable("p")
    a = nm.constraint.variable("a")
    # variables
    x_bar = nm.constraint.variable("x_bar")
    x_rnd = nm.constraint.variable("x_rnd")

    # model
    obj_bar, constrs_bar = probRastrigin(x_bar, p, a, num_vars=num_vars, alpha=100)
    obj_rnd, constrs_rnd = probRastrigin(x_rnd, p, a, num_vars=num_vars, alpha=100)

    # define neural architecture for the solution mapping
    func = nm.modules.blocks.MLP(insize=num_vars+1, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU, hsizes=[80]*4)
    # solution map from model parameters: sol_map(p) -> x
    sol_map = nm.system.Node(func, ["p", "a"], ["x_bar"], name="smap")

    # penalty loss for mapping
    components = [sol_map]
    loss = nm.loss.PenaltyLoss(obj_bar, constrs_bar)
    problem = nm.problem.Problem(components, loss)

    # penalty loss for rounding
    components = [sol_map, round_module]
    loss = nm.loss.PenaltyLoss(obj_rnd, constrs_rnd)
    problem_rnd = nm.problem.Problem(components, loss)

    return problem, problem_rnd

## Exact Solver

In [8]:
from problem.solver import exactRastrigin
model = exactRastrigin(n_vars=num_vars, n_integers=num_ints)

In [9]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model.setParamValue(p, *a)
    tick = time.time()
    xval, objval = model.solve("scip")
    tock = time.time()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [01:49<00:00,  9.13it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000            1000.0   1000.000000
mean      5.878000               0.0      0.108774
std       1.317897               0.0      0.041968
min       4.000000               0.0      0.072309
25%       5.000000               0.0      0.089265
50%       6.000000               0.0      0.091145
75%       7.000000               0.0      0.106765
max       8.000000               0.0      0.281110


In [10]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"[2.0, 0.0, 1.0, 0.0, 1.0]",6.0,0.0,0.281110
1,"[0.0, 0.0, 1.0, 0.0, 2.0]",5.0,0.0,0.092675
2,"[2.0, 0.0, 1.0, 0.0, 1.0]",6.0,0.0,0.106111
3,"[0.0, 1.0, 1.0, 1.0, 2.0]",7.0,0.0,0.090138
4,"[0.0, 1.0, 1.0, 1.0, 2.0]",7.0,0.0,0.139288
...,...,...,...,...
995,"[1.0, 1.0, 0.0, 1.0, 1.0]",4.0,0.0,0.073948
996,"[0.0, 0.0, 1.0, 0.0, 2.0]",5.0,0.0,0.105913
997,"[0.0, 0.0, 1.0, 0.0, 2.0]",5.0,0.0,0.091118
998,"[0.0, 0.0, 1.0, 0.0, 2.0]",5.0,0.0,0.091660


## Heuristic

In [11]:
from heuristic import naive_round

In [12]:
# relaxed model
model_rel = model.relax()

In [13]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    model.setParamValue(p, *a)
    model_rel.setParamValue(p, *a)
    tick = time.time()
    xval_init, _ = model_rel.solve("scip", max_iter=100)
    naive_round(xval_init, model)
    tock = time.time()
    xval, objval = model.getVal()
    sols.append(list(xval.values()))
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [33:34<00:00,  2.01s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      5.378000          0.112628      2.013448
std       1.292589          0.148134     19.919316
min       3.000000          0.000000      0.366162
25%       4.000000          0.000000      1.116277
50%       5.000000          0.000143      1.329107
75%       6.000000          0.227773      1.542366
max       8.000000          0.510274    631.120111


In [14]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"[0, -1, 2, 1, 0]",6.0,0.000000,1.317917
1,"[-1, 0, 0, 0, 2]",5.0,0.000000,1.204374
2,"[1, -1, -1, 1, 1]",5.0,0.123591,1.323012
3,"[-1, -2, 0, 0, 1]",6.0,0.225262,1.100195
4,"[2, 1, -1, 0, 1]",7.0,0.000000,0.898111
...,...,...,...,...
995,"[0, -1, -1, -1, 0]",3.0,0.269571,0.890250
996,"[1, 0, -1, 1, 1]",4.0,0.023561,0.887723
997,"[2, 0, 0, 0, 0]",4.0,0.167210,0.991105
998,"[0, 0, 2, 0, -1]",5.0,0.000000,0.994902


## Learning to Round

In [15]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.int_ind}, name="round")
_, problem = getNMProb(round_func)

In [16]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 40.742042541503906
epoch: 1  train_loss: 36.44521713256836
epoch: 2  train_loss: 43.06996154785156
epoch: 3  train_loss: 47.03972244262695
epoch: 4  train_loss: 45.753028869628906
epoch: 5  train_loss: 46.43376922607422
epoch: 6  train_loss: 48.07091522216797
epoch: 7  train_loss: 45.602291107177734
epoch: 8  train_loss: 43.758934020996094
epoch: 9  train_loss: 46.51628112792969
epoch: 10  train_loss: 37.776885986328125
epoch: 11  train_loss: 44.985107421875
epoch: 12  train_loss: 50.42340087890625
epoch: 13  train_loss: 42.19470977783203
epoch: 14  train_loss: 45.432186126708984
epoch: 15  train_loss: 31.21142578125
epoch: 16  train_loss: 40.188018798828125
epoch: 17  train_loss: 30.037540435791016
epoch: 18  train_loss: 28.244909286499023
epoch: 19  train_loss: 33.26726531982422
epoch: 20  train_loss: 36.31312942504883
epoch: 21  train_loss: 45.448524475097656
epoch: 22  train_loss: 50.75492477416992
epoch: 23  train_loss: 46.395694732666016
epoch: 24  train_los

In [17]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 139.86it/s]


       Obj Val  Constraints Viol  Elapsed Time
count   1000.0       1000.000000   1000.000000
mean       3.0          2.347360      0.006433
std        0.0          1.291684      0.001273
min        3.0          0.002752      0.004000
25%        3.0          1.246561      0.005512
50%        3.0          2.393041      0.006503
75%        3.0          3.454771      0.007010
max        3.0          4.499422      0.010529


In [18]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,2.905608,0.010529
1,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,1.810883,0.008532
2,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,2.123591,0.008531
3,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,3.225262,0.008520
4,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,3.943377,0.007000
...,...,...,...,...
995,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,0.269571,0.005510
996,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,1.023561,0.004854
997,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,1.167210,0.004688
998,"(0.0, 1.0, 1.0, 0.0, 1.0)",3.0,1.740112,0.004999


## Learnable Threshold

In [19]:
from model.layer import netFC
from model.threshold import roundThresholdModel
# round x
layers_rnd = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundThresholdModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                                 int_ind={"x_bar":model.int_ind}, name="round")
_, problem = getNMProb(round_func)

In [20]:
# training
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 59.326900482177734
epoch: 1  train_loss: 41.29281234741211
epoch: 2  train_loss: 39.73933029174805
epoch: 3  train_loss: 41.40693664550781
epoch: 4  train_loss: 41.0604133605957
epoch: 5  train_loss: 41.2647819519043
epoch: 6  train_loss: 41.20734786987305
epoch: 7  train_loss: 40.80793380737305
epoch: 8  train_loss: 37.508853912353516
epoch: 9  train_loss: 43.864646911621094
epoch: 10  train_loss: 39.26064682006836
epoch: 11  train_loss: 39.99040222167969
epoch: 12  train_loss: 38.344642639160156
epoch: 13  train_loss: 41.38597106933594
epoch: 14  train_loss: 40.20343017578125
epoch: 15  train_loss: 37.5882453918457
epoch: 16  train_loss: 39.66452407836914
epoch: 17  train_loss: 41.60590362548828
epoch: 18  train_loss: 41.98244094848633
epoch: 19  train_loss: 40.917606353759766
epoch: 20  train_loss: 42.18752670288086
epoch: 21  train_loss: 42.41547393798828
epoch: 22  train_loss: 41.26091766357422
epoch: 23  train_loss: 45.06894302368164
epoch: 24  train_loss: 4

In [21]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 136.83it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      4.813000          0.750917      0.006575
std       0.390107          0.831895      0.001366
min       4.000000          0.000000      0.003999
25%       5.000000          0.000000      0.005525
50%       5.000000          0.393041      0.006513
75%       5.000000          1.454771      0.007341
max       5.000000          2.499422      0.012532


In [22]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,0.905608,0.010623
1,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,0.000000,0.007516
2,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,0.123591,0.008512
3,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,1.225262,0.007504
4,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,1.943377,0.006004
...,...,...,...,...
995,"(0.0, 0.0, 0.0, -2.0, 0.0)",4.0,0.000000,0.004000
996,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,0.000000,0.005527
997,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,0.000000,0.006513
998,"(0.0, 0.0, -1.0, -2.0, 0.0)",5.0,0.000000,0.007543


## Learning to Round with Fixed Solution Mapping

In [23]:
from model.layer import netFC
from model.round import roundModel
# round x
layers_rnd = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)
round_func = roundModel(layers=layers_rnd, param_keys=["p", "a"], var_keys=["x_bar"], output_keys=["x_rnd"],
                        int_ind={"x_bar":model.int_ind}, name="round")
problem_rel, problem_rnd = getNMProb(round_func)

In [24]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 200.56048583984375
epoch: 1  train_loss: 200.05564880371094
epoch: 2  train_loss: 200.00062561035156
epoch: 3  train_loss: 199.98989868164062
epoch: 4  train_loss: 200.00938415527344
epoch: 5  train_loss: 200.00672912597656
epoch: 6  train_loss: 199.95262145996094
epoch: 7  train_loss: 199.96778869628906
epoch: 8  train_loss: 199.99769592285156
epoch: 9  train_loss: 200.0087127685547
epoch: 10  train_loss: 200.02859497070312
epoch: 11  train_loss: 199.96282958984375
epoch: 12  train_loss: 199.886962890625
epoch: 13  train_loss: 199.97325134277344
epoch: 14  train_loss: 199.9023895263672
epoch: 15  train_loss: 199.96127319335938
epoch: 16  train_loss: 199.89193725585938
epoch: 17  train_loss: 200.06082153320312
epoch: 18  train_loss: 200.04470825195312
epoch: 19  train_loss: 199.99281311035156
epoch: 20  train_loss: 199.95962524414062
epoch: 21  train_loss: 199.9354248046875
epoch: 22  train_loss: 199.9658966064453
epoch: 23  train_loss: 199.99880981445312
epoch: 2

In [25]:
# freeze sol mapping
#problem_rel.freeze()
# training for rounding
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rnd, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 113.4201889038086
epoch: 1  train_loss: 111.99301147460938
epoch: 2  train_loss: 112.25984191894531
epoch: 3  train_loss: 113.07791900634766
epoch: 4  train_loss: 112.0425033569336
epoch: 5  train_loss: 113.03713989257812
epoch: 6  train_loss: 113.28978729248047
epoch: 7  train_loss: 112.51097106933594
epoch: 8  train_loss: 110.89220428466797
epoch: 9  train_loss: 110.82777404785156
epoch: 10  train_loss: 110.6501693725586
epoch: 11  train_loss: 112.83306884765625
epoch: 12  train_loss: 112.2095947265625
epoch: 13  train_loss: 112.64315795898438
epoch: 14  train_loss: 112.79176330566406
epoch: 15  train_loss: 113.62065887451172
epoch: 16  train_loss: 111.35653686523438
epoch: 17  train_loss: 113.41866302490234
epoch: 18  train_loss: 112.29954528808594
epoch: 19  train_loss: 111.8309326171875
epoch: 20  train_loss: 114.70136260986328
epoch: 21  train_loss: 113.65640258789062
epoch: 22  train_loss: 113.66444396972656
epoch: 23  train_loss: 112.0245590209961
epoch: 2

In [26]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_rnd(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 138.90it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      2.266000          3.081360      0.006489
std       0.640042          1.120670      0.001113
min       1.000000          0.045919      0.003995
25%       2.000000          2.243437      0.005653
50%       2.000000          3.021045      0.006512
75%       3.000000          3.992779      0.007010
max       3.000000          5.490592      0.010534


In [27]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(1.0, 0.0, 1.0, 0.0, 0.0)",2.0,3.905608,0.009708
1,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.0,1.810883,0.008515
2,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.0,2.123591,0.009517
3,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.0,3.225262,0.007536
4,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.0,3.943377,0.008521
...,...,...,...,...
995,"(1.0, 0.0, 0.0, 0.0, 0.0)",1.0,2.269571,0.005000
996,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.0,1.023561,0.005681
997,"(1.0, 0.0, 1.0, 0.0, 0.0)",2.0,2.167210,0.006510
998,"(1.0, 0.0, 1.0, 0.0, 1.0)",3.0,1.740112,0.007068


### Learning to Feasibility Pump

In [28]:
from model.layer import netFC
# define neural architecture for the solution mapping
sol_func = nm.modules.blocks.MLP(insize=num_vars+1, outsize=num_vars, bias=True,
                                 linear_map=nm.slim.maps["linear"], nonlin=nn.ReLU,
                                 hsizes=[80]*4)
# define neural architecture for rounding
rnd_layer = netFC(input_dim=num_vars*2+1, hidden_dims=[80]*4, output_dim=num_vars)

In [29]:
# function to get nm optimization model
from problem.neural import probRastrigin
def getProb(x, p, a):
    return probRastrigin(x, p, a, num_vars=num_vars, alpha=100)

In [30]:
from model import feasibilityPumpModel
# feasibility pump model
problem_rel, problem_fp = feasibilityPumpModel(["p", "a"], getProb, sol_func, rnd_layer, int_ind=model.int_ind, num_iters=3)

In [31]:
# training for mapping
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem_rel.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_rel, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 201.03836059570312
epoch: 1  train_loss: 200.00315856933594
epoch: 2  train_loss: 200.04458618164062
epoch: 3  train_loss: 199.9217987060547
epoch: 4  train_loss: 200.03433227539062
epoch: 5  train_loss: 199.97879028320312
epoch: 6  train_loss: 200.00428771972656
epoch: 7  train_loss: 200.03594970703125
epoch: 8  train_loss: 199.9923095703125
epoch: 9  train_loss: 200.03807067871094
epoch: 10  train_loss: 199.88279724121094
epoch: 11  train_loss: 200.01612854003906
epoch: 12  train_loss: 200.03578186035156
epoch: 13  train_loss: 199.96961975097656
epoch: 14  train_loss: 199.97190856933594
epoch: 15  train_loss: 200.0224609375
epoch: 16  train_loss: 199.9395751953125
epoch: 17  train_loss: 199.98155212402344
epoch: 18  train_loss: 199.98475646972656
epoch: 19  train_loss: 200.0723419189453
epoch: 20  train_loss: 200.00851440429688
epoch: 21  train_loss: 199.98455810546875
epoch: 22  train_loss: 200.01275634765625
epoch: 23  train_loss: 199.97805786132812
epoch: 24 

In [32]:
# freeze sol mapping
#problem_rel.freeze()
# training for feasibility pump
lr = 0.001    # step size for gradient descent
epochs = 400  # number of training epochs
warmup = 50   # number of epochs to wait before enacting early stopping policy
patience = 50 # number of epochs with no improvement in eval metric to allow before early stopping
# set adamW as optimizer
optimizer = torch.optim.AdamW(problem.parameters(), lr=lr)
# define trainer
trainer = nm.trainer.Trainer(problem_fp, loader_train, loader_dev, loader_test,
                             optimizer, epochs=epochs, patience=patience, warmup=warmup)
best_model = trainer.train()

epoch: 0  train_loss: 1197.4468994140625
epoch: 1  train_loss: 1198.0772705078125
epoch: 2  train_loss: 1200.3199462890625
epoch: 3  train_loss: 1182.4595947265625
epoch: 4  train_loss: 1193.3525390625
epoch: 5  train_loss: 1188.619384765625
epoch: 6  train_loss: 1199.23779296875
epoch: 7  train_loss: 1195.2188720703125
epoch: 8  train_loss: 1213.049072265625
epoch: 9  train_loss: 1200.4154052734375
epoch: 10  train_loss: 1204.30322265625
epoch: 11  train_loss: 1214.9918212890625
epoch: 12  train_loss: 1202.0118408203125
epoch: 13  train_loss: 1195.0147705078125
epoch: 14  train_loss: 1201.7613525390625
epoch: 15  train_loss: 1202.967529296875
epoch: 16  train_loss: 1193.233154296875
epoch: 17  train_loss: 1204.9725341796875
epoch: 18  train_loss: 1196.9180908203125
epoch: 19  train_loss: 1197.660400390625
epoch: 20  train_loss: 1203.8084716796875
epoch: 21  train_loss: 1200.6768798828125
epoch: 22  train_loss: 1199.1092529296875
epoch: 23  train_loss: 1198.511962890625
epoch: 24  trai

In [33]:
sols, objvals, conviols, elapseds = [], [], [], []
for p, a in tqdm(list(zip(p_test, a_test))):
    datapoints = {"p": torch.tensor(np.array([p]), dtype=torch.float32), "a":torch.tensor(np.array([a]), dtype=torch.float32), "name": "test"}
    tick = time.time()
    output = problem_fp(datapoints)
    tock = time.time()
    x = output["test_x_rnd"]
    # get values
    model.setParamValue(p, *a)
    for ind in model.x:
        model.x[ind].value = x[0, ind].item()
    xval, objval = model.getVal()
    sols.append(xval.values())
    objvals.append(objval)
    conviols.append(sum(model.calViolation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [02:14<00:00,  7.43it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     12.797000          2.340486      0.132399
std       1.394179          2.058503      0.018744
min       7.000000          0.000000      0.116246
25%      13.000000          0.067266      0.120015
50%      13.000000          2.407362      0.124788
75%      14.000000          3.676876      0.138522
max      15.000000          8.908162      0.279630


In [34]:
df

,Sol,Obj Val,Constraints Viol,Elapsed Time
0,"(-2.0, 1.0, 1.0, -2.0, 2.0)",14.0,2.188784,0.185267
1,"(1.0, 1.0, 1.0, -1.0, 3.0)",13.0,3.378234,0.135124
2,"(-2.0, 1.0, 1.0, -2.0, 2.0)",14.0,3.752818,0.150625
3,"(-1.0, 1.0, 1.0, -1.0, 3.0)",13.0,0.549477,0.142117
4,"(-1.0, 1.0, 1.0, -1.0, 3.0)",13.0,0.000000,0.132024
...,...,...,...,...
995,"(1.0, -1.0, 1.0, -1.0, 2.0)",8.0,1.460858,0.126672
996,"(1.0, -2.0, 1.0, -2.0, 2.0)",14.0,5.952877,0.119656
997,"(1.0, -2.0, 1.0, -2.0, 2.0)",14.0,5.665580,0.122765
998,"(1.0, 1.0, 1.0, -1.0, 3.0)",13.0,3.519776,0.120072
